In [12]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

final_df = pd.read_csv("all_combined_data.csv")

# Define Features (X) and Target (y)
final_df = final_df[final_df["year"].between(2014, 2018)]
final_df = final_df.replace("-", pd.NA)
final_df = final_df.replace('(X)', pd.NA)
final_df= final_df.fillna(0)
final_df = final_df.dropna()
final_df = final_df.replace([np.inf, -np.inf], 0)

X = final_df.drop(columns=[col for col in final_df.columns if ("gentrified" in col or"NAMELSAD_COUNTY_20"in col or "ZCTA" in col or "year" in col or "NAME" in col or "Name" in col)  ])
y = final_df["gentrified_2019"]  # Example: Using gentrification for 2023
X = X.replace('(X)', pd.NA)
# Handle Missing Values
X = X.fillna(0)
y = y.fillna(0)
X.dropna()
X = X.replace([np.inf, -np.inf], 0)
y.dropna()
y = y.replace([np.inf, -np.inf], 0)
# Initialize Model
model = RandomForestClassifier(random_state=42)

# Recursive Feature Elimination with Cross-Validation (RFECV)
rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(5), scoring="accuracy", min_features_to_select=5)
rfecv.fit(X, y)

# Optimal Number of Features
optimal_features = rfecv.n_features_
print("Optimal Number of Features:", optimal_features)

# Selected Features
selected_features = X.columns[rfecv.support_].tolist()
print("Selected Features:", selected_features)

# Plot RFECV Results
plt.figure(figsize=(12, 6))
plt.title("RFECV: Model Performance vs Number of Features")
plt.xlabel("Number of Features")
plt.ylabel("Cross-Validation Accuracy")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

C:\Users\valer\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (25,26,27,28,29,30,32,34,35,36,42,43,44,45,46,47,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,102,105,111,116,117,118,119,120,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,159,160,162,163,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\Users\valer\AppData\Local\Programs\Python\Pyth

MemoryError: Unable to allocate 1.02 MiB for an array with shape (133424,) and data type int64

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

final_df = pd.read_csv("all_combined_data.csv")
final_df = final_df.replace("-", pd.NA)
final_df= final_df.fillna(0)


selected_features = ["DP02_0004PE", "DP02_0006PE", "DP02_0008PE", "DP02_0064PE", "DP03_0062E", "DP03_0004PE", "DP03_0027PE"]

# Step 1: Reshape Data to Have One Target Column
# Convert the years 2016-2023 into rows instead of separate columns
# Step 1: Reshape Data to Have One Target Column
# Convert the years 2016-2023 into rows instead of separate columns
gentrification_years = [f"gentrified_{year}" for year in range(2016, 2024)]

df_melted = final_df.melt(id_vars=["ZCTA", "year"] + selected_features, 
                           value_vars=gentrification_years, 
                           var_name="gentrification_year", value_name="gentrified_status")

# Convert "gentrification_year" to an integer (extract the year from the column name)
df_melted["gentrification_year"] = df_melted["gentrification_year"].str.extract("(\d+)").astype(int)

# Step 2: Split into Training (2016-2020) and Testing (2021-2023)
train_df = df_melted[df_melted["gentrification_year"].between(2016, 2020)]
test_df = df_melted[df_melted["gentrification_year"].between(2021, 2023)]

# Define Features (X) and Target (y)
X_train = train_df[selected_features]
y_train = train_df["gentrified_status"]
X_test = test_df[selected_features]
y_test = test_df["gentrified_status"]

# Step 3: Train Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 4: Make Predictions
y_pred = rf_model.predict(X_test)

# Step 5: Evaluate Model Performance
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


C:\Users\valer\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (25,26,27,28,29,30,32,34,35,36,42,43,44,45,46,47,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,102,105,111,116,117,118,119,120,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,159,160,162,163,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
